<a href="https://colab.research.google.com/github/DanC777/ADA_2/blob/master/GITHUB_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
## Use older version of Keras so that embedding layer works

!pip install keras==2.1.1

In [0]:
# Import all relevent packages and methods


import math
from math import sqrt
import random
from numpy.random import seed
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import special
from sklearn import metrics, preprocessing
from sklearn.preprocessing import  MinMaxScaler, StandardScaler
import tensorflow as tf

import keras
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from keras.layers import Dense, Activation, Flatten, Embedding, Concatenate, Input, Dropout
from keras.models import Sequential, Model

from keras.callbacks import EarlyStopping
from keras.layers.core import Dropout
from keras import regularizers
from sklearn.model_selection import StratifiedKFold


from numpy import array
from numpy import argmax
from keras.utils import to_categorical

from sklearn.datasets import make_regression, make_classification
import statsmodels.formula.api as smf

# Regression


## **Regression - 2 continuous, no cats**

In [0]:
## Function to generate synthetic dataset
def get_data1(observations, sigma):

  x1 = np.random.randint(0,100, size = [observations,1]).astype(float)/10
  x2 = np.random.randint(0,50, size = [observations,1]).astype(float)/10
  
  ## Manually set weights
  w1 = 5
  w2 = 2
  
  # Create error term which adds noise to data
  e = np.random.normal(0, sigma, size = [observations,1]).astype(float) 
  
  
  x = np.hstack((x1,x2))
  y = np.dot(x1**2,w1) + np.dot(x2,w2)  + e
  
  df1 = pd.DataFrame(x)
  df2 = pd.DataFrame(y)
  df = pd.concat([df1, df2], axis = 1)
  
  df.columns = ["x1", "x2",  "y"]

  return(df)

## Function to split dataframe into testing and training sets
def split(df, test_size):
  
  test_size = int(np.round(test_size*df["y"].size))
  n = 1 - test_size
  train = df.iloc[:n-1,:]
  test = df.iloc[n-1:,:]
  return(train, test)




In [0]:
## Function to compare normal regression and neural network. 
## Train NN once and run model on random test set 30 times to get average test accuracy

def compare_regression_data1(observations, sigma, test_size, vector, learn, epochs, rounds):
  
  train_ols_list = []
  test_ols_list = []
  
  
  train_nn_list = []
  test_nn_list = []
  
  df = get_data1(observations = observations, sigma = sigma)
  x = np.array(df.iloc[:, :-1]) 
  y = np.array(df.iloc[:,-1])
  
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size= test_size) ## Split data into training and testing set
  
  ## Normalise data
  scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
  scalarX.fit(X_train)
  
  X_train = scalarX.transform(X_train)
  
  X_test = scalarX.transform(X_test)
  
  
 
  
  def build__regression_model(vector, learn):  ## Allow us to specify the size and number of neurons in the network with 'vector'
      
      model = Sequential()
      
      model.add(Dense(vector[0],  activation = 'relu', ## Fix input layer with relu activation function. Number of input neurons = number of features in the data
                      input_dim=X_train.shape[1]) )
      
      for i in range(1,len(vector)-1):  ## Create loop which adds new layers of the sizes specified in 'vector'
        
         model.add(Dense(vector[i], activation= 'relu'))
         
        
      model.add(Dense(1, activation = "linear")) ## Set final layer with output dim = 1, and linear activation function
      
    
      optimizer = tf.train.AdamOptimizer(learn) 

      model.compile(loss='mse',             #Set loss as MSE, and use adam optimizer
                optimizer=optimizer,
                metrics=['mse'])
      return model
    
    
    
    
  model = build__regression_model(vector = vector, learn = learn)
  
  # Store training stats
  model.fit(X_train,y_train, epochs=epochs,
                    validation_split=0.2, verbose=0)
  
  
  [mse, acc] = model.evaluate(X_train,y_train, verbose=0)
  y_pred_nn = model.predict(X_test).astype(np.float64)
  
  ## Take RMSE as final metric
  train_nn = sqrt(mse)
  
  test_nn = sqrt(metrics.mean_squared_error(y_test, y_pred_nn))
  
  train_nn_list.append(train_nn)
  test_nn_list.append(test_nn)
  
  
  
  ## Test on multiple test sets
  for i in range(rounds):
    ## OLS
    df = get_data1(observations = observations, sigma = sigma)
    train, test = split(df, test_size = test_size)
    
    results = smf.ols(formula = 'y ~ np.power(x1,2) + x2 ', data = sm.add_constant(train)).fit()
    
    train_pred_ols = results.fittedvalues
    train_ols = sqrt(metrics.mean_squared_error(train_pred_ols, train["y"]))
    
    test_pred_ols = results.predict(test)
    test_ols = sqrt(metrics.mean_squared_error(test_pred_ols, test["y"]))
    
    train_ols_list.append(train_ols)
    test_ols_list.append(test_ols)
  
    ## NN

    df = get_data1(observations = observations, sigma = sigma)
    x = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:,-1])
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size= test_size) ## Split data into training and testing set
    
    ## Normalise data
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(X_train)
    
    X_train = scalarX.transform(X_train)
    
    X_test = scalarX.transform(X_test)
    
    [mse, acc] = model.evaluate(X_train,y_train, verbose=0)
    y_pred_nn = model.predict(X_test).astype(np.float64)
    
    ## Take RMSE as final metric
    train_nn = sqrt(mse)
    
    test_nn = sqrt(metrics.mean_squared_error(y_test, y_pred_nn))
    
    train_nn_list.append(train_nn)
    test_nn_list.append(test_nn)
  
  ## Find average MSE and standard deviation of error
  
  average_train_ols = np.mean(train_ols_list)
  average_test_ols = np.mean(test_ols_list)
  test_error_ols = np.std(test_ols_list)
    
  average_train_nn = np.mean(train_nn_list)
  average_test_nn = np.mean(test_nn_list)  
  test_error_nn = np.std(test_nn_list)

  return(average_train_ols, average_test_ols, test_error_ols, average_train_nn, average_test_nn, test_error_nn)

In [26]:
## Repeat this cell varying number of observations and noise

train_ols, test_ols, std_ols, train_nn, test_nn, std_nn = compare_regression_data1(observations = 1000, sigma = 0.01, test_size = 0.33, vector = [1024,64,1], learn = 0.01,epochs = 1000, rounds = 30)

print("Training Error OLS: {}        |  Training Error NN: {}".format(train_ols, train_nn))
print("Testing Error OLS: {}         |  Testing Error NN: {}".format(test_ols, test_nn))
print("Testing STD ols: {}           |  Testing std NN: {}".format(std_ols, std_nn))

print("MSE difference between OLS and NN: {}".format(test_ols - test_nn))

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Training Error OLS: 0.010004254735371256        |  Training Error NN: 2.73401703487481
Testing Error OLS: 0.009865358042343609         |  Testing Error NN: 2.736787835081525
Testing STD ols: 0.00038002079859217157           |  Testing std NN: 0.15808290373253145
MSE difference between OLS and NN: -2.726922477039181


## Regression - 2 continuous, one cat, linear

In [0]:
def get_data2(observations, sigma, representation):
 

  x1 = np.random.randint(0,100, size = [observations*1]).reshape([observations,1])/10  ## Data points with n observations and m features
  x2 = np.random.randint(0,50, size = [observations*1]).reshape([observations,1])/10  ## Data points with n observations and m feature
   
  x_cat = np.random.randint(7, size = [observations,1]).astype(int)  ## Add categorical variable (ranging between 1,7 to represent days of week)
  
  
  ## Convert to dataframe to use "get_dummies" function
  x = np.hstack((x1,x2,  x_cat))
  x_original = pd.DataFrame(x)
  x = pd.DataFrame(x)
  cats = x.iloc[:,-1].astype(object)
  cont = x.iloc[:,:-1]
  cats = pd.get_dummies(cats, drop_first = True)
   
  x = pd.concat([cont,cats], axis = 1)
  
  a = np.array([[2,5,1,1,1,1,7,7]]) ## Set cooeficients manually
  e = np.random.normal(0, sigma, size = [observations,1]).astype(float) ## This term adds 'noise' to the data
  y = np.dot(x,a.T) + e
  
  ## Create dataframe's with one-hot encoding for OLS and without for NN
  df1 = pd.DataFrame(x)
  df2 = pd.DataFrame(y)
  df_nn = pd.concat([x_original, df2], axis = 1)
  df_ols = pd.concat([df1, df2], axis = 1)
  
  df_ols.columns = ["x1", "x2", "cat1","cat2","cat3","cat4","cat5","cat6", "y"]
  df_nn.columns = ["x1","x2","cat","y"]
  
  
  
  
  
  if representation == "ols":
    return(df_ols)
  
  else:
    return(df_nn)

def split(df, test_size):
  
  test_size = int(np.round(test_size*df["y"].size))
  n = 1 - test_size
  train = df.iloc[:n-1,:]
  test = df.iloc[n-1:,:]
  return(train, test)


In [0]:
def compare_regression_data2(observations, sigma, test_size, vector, learn, output, epochs, rounds):
  train_ols_list = []
  test_ols_list = []
  
  
  train_nn_list = []
  test_nn_list = []
  
  #NN
  df = get_data2(observations = observations, sigma = sigma, representation = "nn"  )
  x = np.array(df.iloc[:, :-1])
  y = np.array(df.iloc[:,-1])
  
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=test_size) ## Split data into training and testing set
        
  ## Normalise  
  scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
  scalarX.fit(X_train[:,:-1])
  
  X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
  
  X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
  
  
  # Define the embedding input
  cat_input = Input(shape=(1,)) 
  
  ## Numerical inut
  num_input = Input(shape = (2,))
  
  ## Create embedding layer
  embeding = Embedding(input_dim = 7
                            , output_dim = output, input_length = 1)(cat_input)
  
  ## Flatten embedding layer
  embeding = Flatten()(embeding)
  
  ## Concatenate numerical layer and embedding layer
  combined = keras.layers.concatenate([num_input, embeding])
  
  ## Create normal dense layers
  dense1 = Dense(units=vector[0], activation='relu')(combined)
  dense2 = Dense(units = vector[1], activation='relu')(dense1)
  predictions = Dense(1, activation = 'linear')(dense2)
  
  ## Define final model
  model = Model(inputs=[num_input, cat_input], outputs=predictions)
  
  
  optimizer = tf.train.AdamOptimizer(learn) 
    
  model.compile(loss='mse',             #Set loss as MSE, and use adam optimizer
                    optimizer=optimizer,
                    metrics=['mse'])
  
  
 
    
  # Store training stats
  history = model.fit([X_train[:,:-1], X_train[:,-1]],y_train, epochs=epochs,
                    validation_split=0.2, verbose=0, callbacks = [EarlyStopping(monitor='val_mean_squared_error', patience=200)])
  
  
  
  [mse, acc] = model.evaluate([X_train[:,:-1], X_train[:,-1]],y_train, verbose=0)
  
  train_nn = sqrt(mse)
  
  test_nn_pred = np.round(model.predict([X_test[:,:-1], X_test[:,-1]]).astype(np.float64))
  
  test_nn = sqrt(metrics.mean_squared_error(y_test, test_nn_pred))
      
    
  train_nn_list.append(train_nn)
  test_nn_list.append(test_nn)
  
  ## Test on multiple sets
  for i in range(rounds):
    df = get_data2(observations = observations, sigma = sigma, representation = "ols"  )

    train, test = split(df, test_size = test_size)
    
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(train.iloc[:,:-1])
    
    train.iloc[:,:-1]= scalarX.transform(train.iloc[:,:-1])
    
    test.iloc[:,:-1] = scalarX.transform(test.iloc[:,:-1])
    
    results = smf.ols(formula = 'y ~ x1 + x2 + cat1 + cat2 +cat3 + cat4 + cat5 + cat6', data = sm.add_constant(train)).fit()
    
    train_pred_ols = results.fittedvalues
    train_ols = sqrt(metrics.mean_squared_error(train_pred_ols, train["y"]))
    
    test_pred_ols = results.predict(test)
    test_ols = sqrt(metrics.mean_squared_error(test_pred_ols, test["y"]))
    
    train_ols_list.append(train_ols)
    test_ols_list.append(test_ols)
    
    
    ##NN
    df = get_data2(observations = observations, sigma = sigma, representation = "nn"  )

    x = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:,-1])
    
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=test_size) ## Split data into training and testing set
          
    ## Normalise  
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(X_train[:,:-1])
   
    X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
    
    X_test[:,:-1] = scalarX.transform(X_test[:,:-1])

    
    
    
    
    
    [mse, acc] = model.evaluate([X_train[:,:-1], X_train[:,-1]],y_train, verbose=0)
    
    train_nn = sqrt(mse)
    
    test_nn_pred = np.round(model.predict([X_test[:,:-1], X_test[:,-1]]).astype(np.float64))
    
    test_nn = sqrt(metrics.mean_squared_error(y_test, test_nn_pred))
        
      
    train_nn_list.append(train_nn)
    test_nn_list.append(test_nn)
  
  ## Find average MSE and standard deviation of MSE
  average_train_ols = np.mean(train_ols_list)
  average_test_ols = np.mean(test_ols_list)
  std_ols = np.std(test_ols_list)                                  
    
  average_train_nn = np.mean(train_nn_list)
  average_test_nn = np.mean(test_nn_list)
  std_nn = np.std(test_nn_list)                                  
                                    

  return(average_train_ols, average_test_ols, std_ols,average_train_nn, average_test_nn, std_nn)
    

In [29]:
## Repeat this cell varying number of observations and noise

train_ols, test_ols,std_ols, train_nn, test_nn, std_nn = compare_regression_data2(observations = 1000, sigma = 0.1, test_size = 0.33, vector = [1024,512,64,1], learn = 0.01,output = 4,epochs = 1000, rounds = 30)

print("Training Error OLS: {}        |  Training Error NN: {}".format(train_ols, train_nn))
print("Testing Error OLS: {}         |  Testing Error NN: {}".format(test_ols, test_nn))
print("Testing STD ols: {}           |  Testing std NN: {}".format(std_ols, std_nn))

print("MSE difference between OLS and NN: {}".format(test_ols - test_nn))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.p

Training Error OLS: 0.09951564346721511        |  Training Error NN: 0.36213100843447166
Testing Error OLS: 0.10087915045224409         |  Testing Error NN: 0.46180908799949916
Testing STD ols: 0.004381775842180716           |  Testing std NN: 0.013626319908063364
MSE difference between OLS and NN: -0.36092993754725505


## Regression - 2 continuous, one cat, non-linear

In [0]:
def get_data3(observations, sigma, representation):
 

  x1 = np.random.randint(0,100, size = [observations*1]).reshape([observations,1])/10  ## Data points with n observations and m features
  x2 = np.random.randint(0,50, size = [observations*1]).reshape([observations,1])/10  ## Data points with n observations and m feature
   
  x_cat = np.random.randint(7, size = [observations,1]).astype(int)  ## Add categorical variable (ranging between 1,7 to represent days of week)
  
  
  ## Create dataframe and create one-hot encoding
  x = np.hstack((x1,x2,  x_cat))
  x_original = pd.DataFrame(x)
  x = pd.DataFrame(x)
  cats = x.iloc[:,-1].astype(object)
  cont = x.iloc[:,:-1]
  cats = pd.get_dummies(cats, drop_first = True)
   
  x = pd.concat([cont,cats], axis = 1)
  x = np.array(x)
  cats = np.array(cats)
  
  ## Set coefficients manually
  a = np.array([[1,1,1,1,7,7]])
  e = np.random.normal(0, sigma, size = [observations,1]).astype(float) ## This term adds 'noise' to the data
  w1 = 2
  w2 = 5
  
  ## Create non-linear relationship 
  y = np.dot(x1**2,w1) + np.dot(x2, w2) + np.dot(cats, a.T) + e
  
  df1 = pd.DataFrame(x)
  df2 = pd.DataFrame(y)
  df_nn = pd.concat([x_original, df2], axis = 1)
  df_ols = pd.concat([df1, df2], axis = 1)
  
  
  df_ols.columns = ["x1", "x2", "cat1","cat2","cat3","cat4","cat5","cat6", "y"]
  df_nn.columns = ["x1","x2","cat","y"]
  
  
  
 
  
  if representation == "ols":
    return(df_ols)
  
  else:
    return(df_nn)

def split(df, test_size):
  
  test_size = int(np.round(test_size*df["y"].size))
  n = 1 - test_size
  train = df.iloc[:n-1,:]
  test = df.iloc[n-1:,:]
  return(train, test)


In [0]:
def compare_regression_data3(observations, sigma, test_size, vector, learn, output, epochs, rounds):
  train_ols_list = []
  test_ols_list = []
  
  
  train_nn_list = []
  test_nn_list = []
  
  #NN
  df = get_data3(observations = observations, sigma = sigma, representation = "nn"  )

  x = np.array(df.iloc[:, :-1])
  y = np.array(df.iloc[:,-1])
  
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=test_size) ## Split data into training and testing set
        
  scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
  scalarX.fit(X_train[:,:-1])
  
  X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
  
  X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
  
  
  cat_input = Input(shape=(1,)) 
  
  num_input = Input(shape = (2,))
  
  embeding = Embedding(input_dim = 7
                            , output_dim = output, input_length = 1)(cat_input)
  embeding = Flatten()(embeding)
  
  combined = keras.layers.concatenate([num_input, embeding])
  
  
  
  dense1 = Dense(units=vector[0], activation='relu')(combined)
  dense2 = Dense(units = vector[1], activation='relu')(dense1)
  predictions = Dense(1, activation = 'linear')(dense2)
  
  model = Model(inputs=[num_input, cat_input], outputs=predictions)
  
  
  optimizer = tf.train.AdamOptimizer(learn) 
    
  model.compile(loss='mse',             #Set loss as MSE, and use adam optimizer
                    optimizer=optimizer,
                    metrics=['mse'])
  
  
  
    
  # Store training stats
  history = model.fit([X_train[:,:-1], X_train[:,-1]],y_train, epochs=epochs,
                    validation_split=0.2, verbose=0, callbacks = [EarlyStopping(monitor='val_mean_squared_error', patience=200)])
  
  
  
  
  [mse, acc] = model.evaluate([X_train[:,:-1], X_train[:,-1]],y_train, verbose=0)
  
  train_nn = sqrt(mse)
  
  test_nn_pred = np.round(model.predict([X_test[:,:-1], X_test[:,-1]]).astype(np.float64))
  
  test_nn = sqrt(metrics.mean_squared_error(y_test, test_nn_pred))
      
    
  train_nn_list.append(train_nn)
  test_nn_list.append(test_nn)
  
  for i in range(rounds):
    df = get_data3(observations = observations, sigma = sigma, representation = "ols"  )

    train, test = split(df, test_size = test_size)
    
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(train.iloc[:,:-1])
    
    train.iloc[:,:-1]= scalarX.transform(train.iloc[:,:-1])
    
    test.iloc[:,:-1] = scalarX.transform(test.iloc[:,:-1])
    
    results = smf.ols(formula = 'y ~ np.power(x1,2) + x2 + cat1 + cat2 +cat3 + cat4 + cat5 + cat6', data = sm.add_constant(train)).fit()
    
    train_pred_ols = results.fittedvalues
    train_ols = sqrt(metrics.mean_squared_error(train_pred_ols, train["y"]))
    
    test_pred_ols = results.predict(test)
    test_ols = sqrt(metrics.mean_squared_error(test_pred_ols, test["y"]))
    
    train_ols_list.append(train_ols)
    test_ols_list.append(test_ols)
    
    
    ##NN
    df = get_data3(observations = observations, sigma = sigma, representation = "nn"  )

    x = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:,-1])
    
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=test_size) ## Split data into training and testing set
          
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(X_train[:,:-1])
    
    X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
    
    X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
    
   
    
    
   
    
    [mse, acc] = model.evaluate([X_train[:,:-1], X_train[:,-1]],y_train, verbose=0)
    
    train_nn = sqrt(mse)
    
    test_nn_pred = np.round(model.predict([X_test[:,:-1], X_test[:,-1]]).astype(np.float64))
    
    test_nn = sqrt(metrics.mean_squared_error(y_test, test_nn_pred))
        
      
    train_nn_list.append(train_nn)
    test_nn_list.append(test_nn)
  
  
  
  average_train_ols = np.mean(train_ols_list)
  average_test_ols = np.mean(test_ols_list)
  std_ols = np.std(test_ols_list)                                  
    
  average_train_nn = np.mean(train_nn_list)
  average_test_nn = np.mean(test_nn_list)
  std_nn = np.std(test_nn_list)                                  
                                    

  return(average_train_ols, average_test_ols, std_ols,average_train_nn, average_test_nn, std_nn)
    

In [32]:
## Repeat this cell varying number of observations and noise

train_ols, test_ols, std_ols, train_nn, std_nn, test_nn = compare_regression_data3(observations = 1000, sigma = 1, test_size = 0.33, vector = [1024,64,1], learn = 0.01,output = 4,epochs = 1000, rounds = 30)

print("Training Error OLS: {}        |  Training Error NN: {}".format(train_ols, train_nn))
print("Testing Error OLS: {}         |  Testing Error NN: {}".format(test_ols, test_nn))
print("Testing STD ols: {}           |  Testing std NN: {}".format(std_ols, std_nn))

print("MSE difference between OLS and NN: {}".format(test_ols - test_nn))

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:82: Se

Training Error OLS: 0.9990535335490779        |  Training Error NN: 1.359704235007502
Testing Error OLS: 0.998997223041733         |  Testing Error NN: 0.07038691121288694
Testing STD ols: 0.046246900379678074           |  Testing std NN: 1.4077506483307074
MSE difference between OLS and NN: 0.928610311828846


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Classification

## Classification, 2 continous

In [0]:
def get_cat_data1(observations,  sigma):

  ## Set coeeficients
  x1 = np.random.randint(0,100, size = [observations,1]).astype(float)/10
  x2 = np.random.randint(0,50, size = [observations,1]).astype(float)/10
  w1 = 5
  w2 = 2
  
  e = np.random.normal(0, sigma, size = [observations,1]).astype(float) ## This term adds 'noise' to the data
  
  x = np.hstack((x1,x2))
  y = np.dot(x1**2,w1) + np.dot(x2,w2)  + e
  
  ## Convert output to binary
  scaler = MinMaxScaler()
  y = scaler.fit_transform(y) ## Scale y value
  j = y    
  L = [0 if i < j.mean() else 1 for i in y]  ## Split by mean so that both categories have equal representation
  y = np.array(L).reshape(y.shape[0],1)
  
  df1 = pd.DataFrame(x)
  df2 = pd.DataFrame(y)
  df = pd.concat([df1, df2], axis = 1)
  
  df.columns = ["x1", "x2",  "y"]

  return(df)

def split(df, test_size):
  
  test_size = int(np.round(test_size*df["y"].size))
  n = 1 - test_size
  train = df.iloc[:n-1,:]
  test = df.iloc[n-1:,:]
  return(train, test)




In [0]:
def compare_classification_data1(observations, sigma, test_size,  vector, learn,  epochs, rounds):
  train_ols_list = []
  test_ols_list = []
  
  
  train_nn_list = []
  test_nn_list = []
  
  def build_classification_model(vector, learn):
     model = Sequential()
     
     
     model.add(Dense(vector[0], activation = tf.nn.relu,
                     input_dim=X_train.shape[1]))
     
     for i in range(1,len(vector)-1):
       
        model.add(Dense(vector[i], activation=tf.nn.relu))
        
         
       
     model.add(Dense(1, activation = tf.nn.sigmoid))
     
   
     optimizer = tf.train.GradientDescentOptimizer(learn)
     
     ## Use binary crossentropy as loss function and accuracy as metric
     model.compile(optimizer=tf.train.AdamOptimizer(), 
                 loss= 'binary_crossentropy',
                 metrics=['accuracy'])
     return(model)
  
 
  df = get_cat_data1(observations = observations, sigma = sigma)
  x = np.array(df.iloc[:, :-1])
  y = np.array(df.iloc[:,-1])
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size= test_size) ## Split data into training and testing set
  ## Normalise data
  scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
  scalarX.fit(X_train)
  
  X_train = scalarX.transform(X_train)
  
  X_test = scalarX.transform(X_test)
  
  model = build_classification_model(vector, learn)
   
 
  model.fit(X_train,y_train, epochs=epochs,
                      validation_split=0.2, verbose=0)
  
  [loss, acc] = model.evaluate(X_train,y_train, verbose=0)
  
  
  ## Round predictions to convert them to binary
  test_nn_pred = np.round(model.predict(X_test))
  
  test_nn = metrics.accuracy_score(test_nn_pred, y_test)
       
     
  train_nn_list.append(acc)
  test_nn_list.append(test_nn)
  
  
  for i in range(rounds):
    df = get_cat_data1(observations = observations, sigma = sigma)

    train, test = split(df, test_size = test_size)

    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(train.iloc[:,:-1])
    
    train.iloc[:,:-1]= scalarX.transform(train.iloc[:,:-1])
    
    test.iloc[:,:-1] = scalarX.transform(test.iloc[:,:-1])
    
    
    results = smf.logit(formula = 'y ~ np.power(x1,2) + x2 ', data = sm.add_constant(train)).fit(maxiter = 200, method = 'bfgs')
    
    train_pred_ols = (results.fittedvalues > 0.5).astype(int)
        
        
    test_ols = results.predict(sm.add_constant(test))
    
    ## Round output to convert to binary
    test_ols = (test_ols > 0.5).astype(int).reshape(test_ols.size,1)
    
    
    train_ols = metrics.accuracy_score(train_pred_ols, train["y"])
    
    
    test_ols = metrics.accuracy_score(test_ols, test["y"])

    
    train_ols_list.append(train_ols)
    test_ols_list.append(test_ols)
    


    df = get_cat_data1(observations = observations, sigma = sigma)
    x = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:,-1])
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size= test_size) ## Split data into training and testing set
    
    ## Normalise data
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(X_train)
    
    X_train = scalarX.transform(X_train)
    
    X_test = scalarX.transform(X_test)
   
   
    
    [loss, acc] = model.evaluate(X_train,y_train, verbose=0)
    
    
    ## Round ouput to convert it into binary
    test_nn_pred = np.round(model.predict(X_test))
    
    test_nn = metrics.accuracy_score(test_nn_pred, y_test)

        
      
    train_nn_list.append(acc)
    test_nn_list.append(test_nn)
  
  
  ## Return average accuracy and standard deviation of accuracy
  average_train_ols = np.mean(train_ols_list)
  average_test_ols = np.mean(test_ols_list)
  std_ols = np.std(test_ols_list)
    
  average_train_nn = np.mean(train_nn_list)
  average_test_nn = np.mean(test_nn_list)  
  std_nn = np.std(test_nn_list)

  return(average_train_ols, average_test_ols, std_ols, average_train_nn, average_test_nn, std_nn)
    

In [35]:
## Repeat this cell varying number of observations and noise

train_ols, test_ols, std_ols, train_nn, test_nn, std_nn = compare_classification_data1(observations = 100, sigma = 0.01, test_size = 0.33,  vector = [1024,64,1], learn = 0.01, epochs = 1000, rounds= 30)
print("Training Error OLS: {}        |  Training Error NN: {}".format(train_ols, train_nn))
print("Testing Error OLS: {}         |  Testing Error NN: {}".format(test_ols, test_nn))
print("Testing STD ols: {}           |  Testing std NN: {}".format(std_ols, std_nn))

print("MSE difference between OLS and NN: {}".format(test_ols - test_nn))

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: FutureWarning: reshape is deprecated and will raise in a subs

Optimization terminated successfully.
         Current function value: 0.000009
         Iterations: 48
         Function evaluations: 50
         Gradient evaluations: 50


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Optimization terminated successfully.
         Current function value: 0.000009
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


...

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying t

Optimization terminated successfully.
         Current function value: 0.000006
         Iterations: 39
         Function evaluations: 41
         Gradient evaluations: 41


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Optimization terminated successfully.
         Current function value: 0.000010
         Iterations: 42
         Function evaluations: 44
         Gradient evaluations: 44
Optimization terminated successfully.
         Current function value: 0.000010
         Iterations: 28
         Function evaluations: 30
         Gradient evaluations: 30
Training Error OLS: 1.0        |  Training Error NN: 0.9711121811164245
Testing Error OLS: 0.9797979797979797         |  Testing Error NN: 0.976539589442815
Testing STD ols: 0.023903352658988332           |  Testing std NN: 0.02518885381568938
MSE difference between OLS and NN: 0.00325839035516462


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


## **Classification, 2 continous, one categorical, linear **

In [0]:
def get_cat_data2(observations, sigma, representation):
 

  x1 = np.random.randint(0,100, size = [observations,1])/10  ## Data points with n observations and m features
  x2 = np.random.randint(0,50, size = [observations,1])/10  ## Data points with n observations and m feature
   
  # Add categorical variable (ranging between 1,7 to represent days of week)  
  x_cat = np.random.randint(7, size = [observations,1]).astype(int)  
  
  
  ## Convert to dataframe and apply one-hot encoding
  x = np.hstack((x1,x2,  x_cat))
  x_original = pd.DataFrame(x)
  x = pd.DataFrame(x)
  cats = x.iloc[:,-1].astype(object)
  cont = x.iloc[:,:-1]
  cats = pd.get_dummies(cats, drop_first = True)
   
  x = pd.concat([cont,cats], axis = 1)
  x = np.array(x)
  
  a = np.array([[2,5,1,1,1,1,7,7]])
  e = np.random.normal(0, sigma, size = [observations,1]).astype(float) ## This term adds 'noise' to the data
  
  ## Linear relationship
  y = np.dot(x,a.T) + e
  
  ## Convert output to binary, with equal number in each
  scaler = MinMaxScaler()
  y = scaler.fit_transform(y)
  j = y
    
  L = [0 if i < j.mean() else 1 for i in y]
    
  y = np.array(L).reshape(y.shape[0],1)
  
  df1 = pd.DataFrame(x)
  df2 = pd.DataFrame(y)
  df_nn = pd.concat([x_original, df2], axis = 1)
  df_ols = pd.concat([df1, df2], axis = 1)
  
  df_ols.columns = ["x1", "x2", "cat1","cat2","cat3","cat4","cat5","cat6", "y"]
  df_nn.columns = ["x1","x2","cat","y"]
  
  
  if representation == "ols":
    return(df_ols)
  
  else:
    return(df_nn)

def split(df, test_size):
  
  test_size = int(np.round(test_size*df["y"].size))
  n = 1 - test_size
  train = df.iloc[:n-1,:]
  test = df.iloc[n-1:,:]
  return(train, test)


In [0]:
def compare_classification_data2(observations, sigma, test_size,  vector, output, learn,  epochs, rounds):
  train_ols_list = []
  test_ols_list = []
  
  
  train_nn_list = []
  test_nn_list = []
  
  df= get_cat_data2(observations = observations, sigma = sigma, representation = "nn")
  x = np.array(df.iloc[:, :-1])
  y = np.array(df.iloc[:,-1])
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=test_size) ## Split data into training and testing set
   
  ## Normalise   
  scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
  scalarX.fit(X_train[:,:-1])
  
  X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
  
  X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
  
  # Define the embedding input
  cat_input = Input(shape=(1,), dtype='float32') 
  
  ## Define numerical input
  num_input = Input(shape = (2,), dtype = 'float32')
  
  ## Define embedding layer
  embeding = Embedding(input_dim = 7
                            , output_dim = output, input_length = 1 )(cat_input)
  
  ## Flatten embedding layer
  embeding = Flatten()(embeding)
  
  ## Concatenate embedding with numerical
  combined = keras.layers.concatenate([num_input, embeding])
  
  
  ## Add dense layers
  dense1 = Dense(units=vector[0], activation='relu')(combined)
  dense2 = Dense(units = vector[1], activation='relu')(dense1)
  
  predictions = Dense(1, activation = 'sigmoid')(dense2)
  
  ## Define final model
  model = Model(inputs=[num_input, cat_input], outputs=predictions)
  
  model.compile(optimizer=tf.train.AdamOptimizer(), 
            loss= 'binary_crossentropy',
            metrics=['accuracy'])
  
  
  
  
  # Store training stats
  history = model.fit([X_train[:,:-1], X_train[:,-1]],y_train, epochs=epochs,
                    validation_split=0.2, verbose=0, callbacks = [EarlyStopping(monitor='val_acc', patience=150)])
  
  
   
  
  
  
  [mse, acc] = model.evaluate([X_train[:,:-1], X_train[:,-1]],y_train, verbose=0)
  
  ## Round output to convert to binary
  test_nn_pred = np.round(model.predict([X_test[:,:-1], X_test[:,-1]]).astype(np.float64))
  
       
  
  test_nn = metrics.accuracy_score(y_test, test_nn_pred)
  train_nn_list.append(acc)
  test_nn_list.append(test_nn)
  
  for i in range(rounds):
    df= get_cat_data2(observations = observations, sigma = sigma, representation = "nn")
    train, test = split(df, test_size = test_size)
    
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(train.iloc[:,:-1])
    
    train.iloc[:,:-1]= scalarX.transform(train.iloc[:,:-1])
    
    test.iloc[:,:-1] = scalarX.transform(test.iloc[:,:-1])
    
    results = smf.logit(formula = 'y ~ x1 + x2 + C(cat) ', data = sm.add_constant(train)).fit(method = 'bfgs', maxiter = 200)
    
    train_pred_ols = (results.fittedvalues > 0.5).astype(int)
        
      
        
    test_ols = results.predict(sm.add_constant(test))
    test_ols = (test_ols > 0.5).astype(int).reshape(test_ols.size,1)
    
    
    train_ols = metrics.accuracy_score(train_pred_ols, train["y"])
    
    
    test_ols = metrics.accuracy_score(test_ols, test["y"])
        
    train_ols_list.append(train_ols)
    test_ols_list.append(test_ols)
    
    
    
    
    df= get_cat_data2(observations = observations, sigma = sigma, representation = "nn")
    x = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:,-1])
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=test_size) ## Split data into training and testing set
          
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(X_train[:,:-1])
    
    X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
    
    X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
    

    
    [mse, acc] = model.evaluate([X_train[:,:-1], X_train[:,-1]],y_train, verbose=0)
    
    ## Round output to convert to binary
    test_nn_pred = np.round(model.predict([X_test[:,:-1], X_test[:,-1]]).astype(np.float64))
    
         
    
    test_nn = metrics.accuracy_score(y_test, test_nn_pred)
    train_nn_list.append(acc)
    test_nn_list.append(test_nn)
  
  average_train_ols = np.mean(train_ols_list)
  average_test_ols = np.mean(test_ols_list)
  std_ols = np.std(test_ols_list)
  
  average_train_nn = np.mean(train_nn_list)
  average_test_nn = np.mean(test_nn_list)
  std_nn = np.std(test_nn_list)

  return(average_train_ols, average_test_ols, std_ols, average_train_nn, average_test_nn, std_nn)
    

In [38]:
## Repeat this cell varying number of observations and noise

train_ols, test_ols, std_ols, train_nn, test_nn, std_nn = compare_classification_data2(observations = 200, sigma = 0.01, test_size = 0.33,  vector = [1024,64,1], output = 4, learn = 0.01,  epochs = 1000, rounds = 30)

print("Training Error OLS: {}        |  Training Error NN: {}".format(train_ols, train_nn))
print("Testing Error OLS: {}         |  Testing Error NN: {}".format(test_ols, test_nn))
print("Testing STD ols: {}           |  Testing std NN: {}".format(std_ols, std_nn))

print("MSE difference between OLS and NN: {}".format(test_ols - test_nn))

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


Optimization terminated successfully.
         Current function value: 0.000017
         Iterations: 66
         Function evaluations: 67
         Gradient evaluations: 67


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bs

Optimization terminated successfully.
         Current function value: 0.000022
         Iterations: 71
         Function evaluations: 73
         Gradient evaluations: 73


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


...

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Optimization terminated successfully.
         Current function value: 0.000019
         Iterations: 63
         Function evaluations: 65
         Gradient evaluations: 65


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Optimization terminated successfully.
         Current function value: 0.000010
         Iterations: 75
         Function evaluations: 76
         Gradient evaluations: 76
Training Error OLS: 1.0        |  Training Error NN: 0.9405392391439464
Testing Error OLS: 0.9540404040404039         |  Testing Error NN: 0.9472140762463341
Testing STD ols: 0.0257971571497417           |  Testing std NN: 0.03607554660263136
MSE difference between OLS and NN: 0.006826327794069775


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


## Classification, 2 continous, one categorical, non-linear

In [0]:
def get_cat_data3(observations, sigma,  representation):
 

  x1 = np.random.randint(0,100, size = [observations*1]).reshape([observations,1])/10  ## Data points with n observations and m features
  x2 = np.random.randint(0,50, size = [observations*1]).reshape([observations,1])/10  ## Data points with n observations and m feature
   
  x_cat = np.random.randint(7, size = [observations,1]).astype(int)  ## Add categorical variable (ranging between 1,7 to represent days of week)
  
  
  
  x = np.hstack((x1,x2,  x_cat))
  x_original = pd.DataFrame(x)
  x = pd.DataFrame(x)
  cats = x.iloc[:,-1].astype(object)
  cont = x.iloc[:,:-1]
  cats = pd.get_dummies(cats, drop_first = True)
   
  x = pd.concat([cont,cats], axis = 1)
  x = np.array(x)
  cats = np.array(cats)
  
  a = np.array([[1,1,1,1,7,7]])
  e = np.random.normal(0, sigma, size = [observations,1]).astype(float) ## This term adds 'noise' to the data
  w1 = 2
  w2 = 5
  
  ## Create non-linear relationship
  y = np.dot(x1**2,w1) + np.dot(x2, w2) + np.dot(cats, a.T) + e
  
  scaler = MinMaxScaler()
  y = scaler.fit_transform(y)
  j = y
    
  L = [0 if i < j.mean() else 1 for i in y]
    
  y = np.array(L).reshape(y.shape[0],1)
  
  df1 = pd.DataFrame(x)
  df2 = pd.DataFrame(y)
  df_nn = pd.concat([x_original, df2], axis = 1)
  df_ols = pd.concat([df1, df2], axis = 1)
  
  
  df_ols.columns = ["x1", "x2", "cat1","cat2","cat3","cat4","cat5","cat6", "y"]
  df_nn.columns = ["x1","x2","cat","y"]
  
  
  
 
  
  if representation == "ols":
    return(df_ols)
  
  else:
    return(df_nn)

def split(df, test_size):
  
  test_size = int(np.round(test_size*df["y"].size))
  n = 1 - test_size
  train = df.iloc[:n-1,:]
  test = df.iloc[n-1:,:]
  return(train, test)



In [0]:
def compare_classification_data3(observations, sigma, test_size,  vector, output, learn,  epochs, rounds):
  train_ols_list = []
  test_ols_list = []
  
  
  train_nn_list = []
  test_nn_list = []
  
  df= get_cat_data3(observations = observations, sigma = sigma, representation = "nn")
    
  x = np.array(df.iloc[:, :-1])
  y = np.array(df.iloc[:,-1])
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=test_size) ## Split data into training and testing set
  
  ## Normalise
  scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
  scalarX.fit(X_train[:,:-1])
  
  X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
  
  X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
  
  # Define the embedding input
  cat_input = Input(shape=(1,), dtype='float32') 
  
  ## Define numerical input
  num_input = Input(shape = (2,), dtype = 'float32')
  
  ## Create embedding layer
  embeding = Embedding(input_dim = 7
                            , output_dim = output, input_length = 1 )(cat_input)
  embeding = Flatten()(embeding)
  
  combined = keras.layers.concatenate([num_input, embeding])
  
  ## Create normal layers
  
  dense1 = Dense(units=vector[0], activation='relu')(combined)
  dense2 = Dense(units = vector[1], activation='relu')(dense1)
  predictions = Dense(1, activation = 'sigmoid')(dense2)
  
  ## Define final model
  model = Model(inputs=[num_input, cat_input], outputs=predictions)
  model.compile(optimizer=tf.train.AdamOptimizer(), 
            loss= 'binary_crossentropy',
            metrics=['accuracy'])
  
  
  
  # Store training stats
  history = model.fit([X_train[:,:-1], X_train[:,-1]],y_train, epochs=epochs,
                    validation_split=0.2, verbose=0, callbacks = [EarlyStopping(monitor='val_acc', patience=150)])
  
  

  
  [mse, acc] = model.evaluate([X_train[:,:-1], X_train[:,-1]],y_train, verbose=0)
    
    
  test_nn_pred = np.round(model.predict([X_test[:,:-1], X_test[:,-1]]).astype(np.float64))
  
  
       
  
  test_nn = metrics.accuracy_score(y_test, test_nn_pred)
  train_nn_list.append(acc)
  test_nn_list.append(test_nn)
  
  for i in range(rounds):
    df = get_cat_data3(observations = observations, sigma = sigma,  representation = "nn")
    train, test = split(df, test_size = test_size)
    
   
    
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(train.iloc[:,:-1])
    
    train.iloc[:,:-1]= scalarX.transform(train.iloc[:,:-1])
    
    test.iloc[:,:-1] = scalarX.transform(test.iloc[:,:-1])
    
    results = smf.logit(formula = 'y ~ np.power(x1,2) + x2 + C(cat) ', data = sm.add_constant(train)).fit(method = 'bfgs', maxiter = 200,verbose = 0)
    
    train_pred_ols = (results.fittedvalues > 0.5).astype(int)
        
        
    test_ols = results.predict(sm.add_constant(test))
    test_ols = (test_ols > 0.5).astype(int).reshape(test_ols.size,1)
    
    
    train_ols = metrics.accuracy_score(train_pred_ols, train["y"])
    
    
    test_ols = metrics.accuracy_score(test_ols, test["y"])
            
    train_ols_list.append(train_ols)
    test_ols_list.append(test_ols)
    
    
    
    
    df= get_cat_data3(observations = observations, sigma = sigma, representation = "nn")
    
    x = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:,-1])
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=test_size) ## Split data into training and testing set
    
    ## Normalise
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(X_train[:,:-1])
    
    X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
    
    X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
    

    
    [mse, acc] = model.evaluate([X_train[:,:-1], X_train[:,-1]],y_train, verbose=0)
      
      
    test_nn_pred = np.round(model.predict([X_test[:,:-1], X_test[:,-1]]).astype(np.float64))

    
    
    test_nn = metrics.accuracy_score(y_test, test_nn_pred)
    train_nn_list.append(acc)
    test_nn_list.append(test_nn)
  
  average_train_ols = np.mean(train_ols_list)
  average_test_ols = np.mean(test_ols_list)
  std_ols = np.std(test_ols_list)                        
    
  average_train_nn = np.mean(train_nn_list)
  average_test_nn = np.mean(test_nn_list)
  std_nn = np.std(test_nn_list)

  return(average_train_ols, average_test_ols,std_ols, average_train_nn, average_test_nn, std_nn)
    

In [41]:
## Repeat this cell varying number of observations and noise

train_ols, test_ols,std_ols, train_nn, test_nn, std_nn = compare_classification_data3(observations = 200, sigma = 0.1, test_size = 0.33,  vector = [1024,64,1], output = 4, learn = 0.01,  epochs = 1000, rounds = 30)

print("Training Error OLS: {}        |  Training Error NN: {}".format(train_ols, train_nn))
print("Testing Error OLS: {}         |  Testing Error NN: {}".format(test_ols, test_nn))
print("Testing STD ols: {}           |  Testing std NN: {}".format(std_ols, std_nn))

print("MSE difference between OLS and NN: {}".format(test_ols - test_nn))

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


Optimization terminated successfully.
         Current function value: 0.000007
         Iterations: 53
         Function evaluations: 54
         Gradient evaluations: 54


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Optimization terminated successfully.
         Current function value: 0.000008
         Iterations: 38
         Function evaluations: 40
         Gradient evaluations: 40


/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

...

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bs

Optimization terminated successfully.
         Current function value: 0.000025
         Iterations: 82
         Function evaluations: 84
         Gradient evaluations: 84


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying t

Optimization terminated successfully.
         Current function value: 0.000009
         Iterations: 35
         Function evaluations: 37
         Gradient evaluations: 37
Optimization terminated successfully.
         Current function value: 0.000041
         Iterations: 38
         Function evaluations: 39
         Gradient evaluations: 39
Training Error OLS: 1.0        |  Training Error NN: 0.9330765524906898
Testing Error OLS: 0.968181818181818         |  Testing Error NN: 0.9349951124144672
Testing STD ols: 0.022592191157924275           |  Testing std NN: 0.0346777551809712
MSE difference between OLS and NN: 0.03318670576735083


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


# No Embedding

## Regression

### Regression, data2

In [0]:
def NN_regression_2(observations, sigma, vector,   rounds):

  train = []
  test = []
  
  vector = vector
    
  learn = 0.01
   
  
  
  
  df = get_data2(observations = observations, sigma = sigma, representation = "ols"  )
  
  
  x = np.array(df.iloc[:, :-1])
  y = np.array(df.iloc[:,-1])
  
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33) ## Split data into training and testing set
    
  ## Normalise  
  scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
  scalarX.fit(X_train[:,:-1])

  X_train[:,:-1] = scalarX.transform(X_train[:,:-1])

  X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
  
  
  def build__regression_model(vector, learn):  ## Allow us to specify the size and number of neurons in the network with 'vector'
      
      model = Sequential()
      
      model.add(Dense(vector[0],   ## Fix input layer with relu activation function. Number of input neurons = number of features in the data
                      input_dim=X_train.shape[1]) )
      
      for i in range(1,len(vector)-1):  ## Create loop which adds new layers of the sizes specified in 'vector'
        
         model.add(Dense(vector[i], activation=tf.nn.relu))
         
        
      model.add(Dense(1, activation = "linear")) ## Set final layer with output dim = 1, and linear activation function
      
    
      optimizer = tf.train.AdamOptimizer(learn) 
    
      model.compile(loss='mse',             #Set loss as MSE, and use adam optimizer
                    optimizer=optimizer,
                    metrics=['mse'])
      return model
    
    
    
    
  model = build__regression_model(vector = vector, learn = learn)
  EPOCHS = 1000
  
  # Store training stats
  history = model.fit(X_train,y_train, epochs=EPOCHS,
                    validation_split=0.2, verbose=0)
  
  
 
  
  [mse, acc] = model.evaluate(X_train,y_train, verbose=0)
  y_pred_nn = model.predict(X_test).astype(np.float64)
  
  
  train_nn = sqrt(mse)
  
  train.append(train_nn)
  
  test_nn = sqrt(metrics.mean_squared_error(y_test, y_pred_nn))
  test.append(test_nn)
  for i in range(rounds):
    
    
    
    df = get_data2(observations = observations, sigma = sigma, representation = "ols"  )
    
    
    x = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:,-1])
    #
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33) ## Split data into training and testing set
    
    ## Normalise
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(X_train[:,:-1])

    X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
 
    X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
    
    
    
    
    [mse, acc] = model.evaluate(X_train,y_train, verbose=0)
    y_pred_nn = model.predict(X_test).astype(np.float64)
    
    
    train_nn = sqrt(mse)
    
    train.append(train_nn)
    
    test_nn = sqrt(metrics.mean_squared_error(y_test, y_pred_nn))
    test.append(test_nn)
    
  average_train = np.mean(train)
  average_test = np.mean(test)
  std = np.std(test)
  return(average_train, average_test, std)
  

In [43]:
## Repeat this cell varying number of observations and noise

train, test, std = NN_regression_2(observations = 1000, sigma = 0.01, vector = [1024,64,1], rounds = 30)

print("Train: {}  |  Test: {}     | std: {}".format(train, test, std))

Train: 0.07373493671151474  |  Test: 0.07303772513676275     | std: 0.0025677670291199676


### Regression, data3

In [0]:
def NN_regression_3(observations, sigma, vector,   rounds):

  train = []
  test = []
  
  vector = vector
  
  learn = 0.01
   
  
  
  
  df = get_data3(observations = observations, sigma = sigma, representation = "ols"  )
  
  
  x = np.array(df.iloc[:, :-1])
  y = np.array(df.iloc[:,-1])
  
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33) ## Split data into training and testing set
  
  ## Normalise
  scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
  scalarX.fit(X_train[:,:-1])
  
  X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
  
  X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
  
  
  def build__regression_model(vector, learn):  ## Allow us to specify the size and number of neurons in the network with 'vector'
      
      model = Sequential()
      
      model.add(Dense(vector[0],   ## Fix input layer with relu activation function. Number of input neurons = number of features in the data
                      input_dim=X_train.shape[1]) )
      
      for i in range(1,len(vector)-1):  ## Create loop which adds new layers of the sizes specified in 'vector'
        
         model.add(Dense(vector[i], activation=tf.nn.relu))
         
        
      model.add(Dense(1, activation = "linear")) ## Set final layer with output dim = 1, and linear activation function
      
    
      optimizer = tf.train.AdamOptimizer(learn) 
    
      model.compile(loss='mse',             #Set loss as MSE, and use adam optimizer
                    optimizer=optimizer,
                    metrics=['mse'])
      return model
    
    
    
    
  model = build__regression_model(vector = vector, learn = learn)
  EPOCHS = 1000
  
  # Store training stats
  history = model.fit(X_train,y_train, epochs=EPOCHS,
                    validation_split=0.2, verbose=0)
  
  
   
  
  [mse, acc] = model.evaluate(X_train,y_train, verbose=0)
  y_pred_nn = model.predict(X_test).astype(np.float64)
  
  
  train_nn = sqrt(mse)
  
  train.append(train_nn)
  
  test_nn = sqrt(metrics.mean_squared_error(y_test, y_pred_nn))
  test.append(test_nn)
  for i in range(rounds):
   
   
    
    
    df = get_data3(observations = observations, sigma = sigma, representation = "ols"  )
    
    
    x = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:,-1])
    
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33) ## Split data into training and testing set
     
    # Normalise  
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(X_train[:,:-1])
   
    X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
  
    X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
    
    
    
    
    [mse, acc] = model.evaluate(X_train,y_train, verbose=0)
    y_pred_nn = model.predict(X_test).astype(np.float64)
    
    
    train_nn = sqrt(mse)
    
    train.append(train_nn)
    
    test_nn = sqrt(metrics.mean_squared_error(y_test, y_pred_nn))
    test.append(test_nn)
    
    
  average_train = np.mean(train)
  average_test = np.mean(test)
  std = np.std(test)
  return(average_train, average_test, std)
  

In [45]:
## Repeat this cell varying number of observations and noise
train, test, std = NN_regression_3(observations = 1000, sigma = 0.01, vector = [1024,64,1], rounds = 30)

print("Train: {}  |  Test: {}     | std: {}".format(train, test, std))

Train: 0.7554924980650136  |  Test: 0.7742341695003457     | std: 0.03629413108886143


## Classification

### Classification data2

In [0]:
def NN_classification_2(observations, sigma, vector,   rounds):

  train = []
  test = []
  
  vector = vector
    
  learn = 0.01
   
  
  
  
  df = get_cat_data2(observations = observations, sigma = sigma, representation = "ols"  )
  
  
  x = np.array(df.iloc[:, :-1])
  y = np.array(df.iloc[:,-1])
  
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33) ## Split data into training and testing set
   
  ## Normalise  
  scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
  scalarX.fit(X_train[:,:-1])
  
  X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
  
  X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
  
  
  def build__classification_model(vector, learn):  ## Allow us to specify the size and number of neurons in the network with 'vector'
      
      model = Sequential()
      
      model.add(Dense(vector[0],   ## Fix input layer with relu activation function. Number of input neurons = number of features in the data
                      input_dim=X_train.shape[1]) )
      
      for i in range(1,len(vector)-1):  ## Create loop which adds new layers of the sizes specified in 'vector'
        
         model.add(Dense(vector[i], activation=tf.nn.relu))
         
        
      model.add(Dense(1, activation = "sigmoid")) ## Set final layer with output dim = 1, and linear activation function
      
    
      optimizer = tf.train.AdamOptimizer(learn) 
    
      model.compile(loss='binary_crossentropy',             #Set loss as MSE, and use adam optimizer
                    optimizer=optimizer,
                    metrics=['accuracy'])
      return model
    
    
    
    
  model = build__classification_model(vector = vector, learn = learn)
  EPOCHS = 1000
  
  # Store training stats
  history = model.fit(X_train,y_train, epochs=EPOCHS,
                    validation_split=0.2, verbose=0)
  
  
 
  
  [loss, acc] = model.evaluate(X_train,y_train, verbose=0)
  test_nn_pred = np.round(model.predict(X_test).astype(np.float64))
  
  test_nn = metrics.accuracy_score(y_test, test_nn_pred)
  
  
  
  
  train.append(acc)
  
  
  test.append(test_nn)
  for i in range(rounds):
    
    
    
    
    df = get_cat_data2(observations = observations, sigma = sigma, representation = "ols"  )
    
    
    x = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:,-1])
    
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33) ## Split data into training and testing set
          
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(X_train[:,:-1])
    
    X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
    
    X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
    
    
    
   
    
    [loss, acc] = model.evaluate(X_train,y_train, verbose=0)


    test_nn_pred = np.round(model.predict(X_test).astype(np.float64))

     

    test_nn = metrics.accuracy_score(y_test, test_nn_pred)
    
    
    
    
    train.append(acc)
    
    
    test.append(test_nn)
    
  average_train = np.mean(train)
  average_test = np.mean(test)
  std = np.std(test)                        
  return(average_train, average_test, std)
  


In [47]:
## Repeat this cell varying number of observations and noise
train, test, std = NN_classification_2(observations = 200, sigma = 0.01, vector = [1024,64,1], rounds = 30)

print("Train: {}  |  Test: {}     | std: {}".format(train, test, std))

Train: 0.9576311986436049  |  Test: 0.9516129032258065     | std: 0.026779330462957433


### Classification data3

In [0]:
def NN_classification_3(observations, sigma, vector,   rounds):

  train = []
  test = []
  
  vector = vector
    
  learn = 0.01
   
  
  
  
  df = get_cat_data3(observations = observations, sigma = sigma, representation = "ols"  )
  
  
  x = np.array(df.iloc[:, :-1])
  y = np.array(df.iloc[:,-1])
  #
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33) ## Split data into training and testing set
  
  ## Normalise
  scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
  scalarX.fit(X_train[:,:-1])

  X_train[:,:-1] = scalarX.transform(X_train[:,:-1])

  X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
  
  
  def build__classification_model(vector, learn):  ## Allow us to specify the size and number of neurons in the network with 'vector'
      
      model = Sequential()
      
      model.add(Dense(vector[0],   ## Fix input layer with relu activation function. Number of input neurons = number of features in the data
                      input_dim=X_train.shape[1]) )
      
      for i in range(1,len(vector)-1):  ## Create loop which adds new layers of the sizes specified in 'vector'
        
         model.add(Dense(vector[i], activation=tf.nn.relu))
         
        
      model.add(Dense(1, activation = "sigmoid")) ## Set final layer with output dim = 1, and linear activation function
      
    
      optimizer = tf.train.AdamOptimizer(learn) 
    
      model.compile(loss='binary_crossentropy',             #Set loss as MSE, and use adam optimizer
                    optimizer=optimizer,
                    metrics=['accuracy'])
      return model
    
    
    
    
  model = build__classification_model(vector = vector, learn = learn)
  EPOCHS = 1000
  
  # Store training stats
  history = model.fit(X_train,y_train, epochs=EPOCHS,
                    validation_split=0.2, verbose=0)
  
  
 
  
  [loss, acc] = model.evaluate(X_train,y_train, verbose=0)
  test_nn_pred = np.round(model.predict(X_test).astype(np.float64))
  
  test_nn = metrics.accuracy_score(y_test, test_nn_pred)
  
  
  
  
  train.append(acc)
  
  
  test.append(test_nn)
    
  for i in range(rounds):
   
    
    
    df = get_cat_data3(observations = observations, sigma = sigma, representation = "ols"  )
    
    
    x = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:,-1])
    #
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33) ## Split data into training and testing set
          
    scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
    scalarX.fit(X_train[:,:-1])
  
    X_train[:,:-1] = scalarX.transform(X_train[:,:-1])
    
    X_test[:,:-1] = scalarX.transform(X_test[:,:-1])
    
    
   
    [loss, acc] = model.evaluate(X_train,y_train, verbose=0)


    test_nn_pred = np.round(model.predict(X_test).astype(np.float64))

     

    test_nn = metrics.accuracy_score(y_test, test_nn_pred)
    
    
    
    
    train.append(acc)
    
    
    test.append(test_nn)
    
  average_train = np.mean(train)
  average_test = np.mean(test)
  std = np.std(test)                        
  return(average_train, average_test,std)
  


In [1]:
## Repeat this cell varying number of observations and noise
train, test, std = NN_classification_3(observations = 200, sigma = 0.01, vector = [1024,64,1], rounds = 30)

print("Train: {}  |  Test: {}     | std: {}".format(train, test, std))

NameError: ignored